In [1]:
!pip3 install selenium


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import queue
import threading
import requests

In [4]:
class ProxyManager:
    def __init__(self):
        self.headers = {
            "Accept": "*/*",
            "Accept-Language": "en-US,en;q=0.5",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
        }

    def get_proxies(self):
        self.proxies = queue.Queue()
        
        url = "https://free-proxy-list.net"
        r = requests.get(url, headers=self.headers)

        soup = BeautifulSoup(r.text, "html.parser")
        table_rows = soup.find("section", attrs={"id": "list"}).find("tbody").find_all("tr")

        for row in table_rows:
            ip_address = row.find_all("td")[0].text.strip()
            port = row.find_all("td")[1].text.strip()

            if row.find_all("td")[6].text.strip() == "yes":
                self.proxies.put(f"{ip_address}:{port}")

    def check_proxies(self):
        self.valid_proxies = []

        while not self.proxies.empty():
            proxy = self.proxies.get()

            try:
                r = requests.get("https://www.google.com", headers=self.headers, proxies={"http": proxy, "https": proxy}, timeout=5)
            except:
                continue

            if r.status_code == 200:
                self.valid_proxies.append(proxy)

    def get_proxy_list(self):
        while True:
            try:
                self.get_proxies()

                threads = [threading.Thread(target=self.check_proxies) for _ in range(32)]

                [t.start() for t in threads]
                [t.join() for t in threads]

                assert len(self.valid_proxies) > 0
            
                return self.valid_proxies
            except:
                continue


In [5]:
def setup_driver(proxy=None):
    chrome_options = webdriver.ChromeOptions()
    if proxy:
        chrome_options.add_argument(f'--proxy-server={proxy}')
    chrome_options.add_argument("--lang=en-US")
    chrome_options.add_argument("--start-maximized")
    chrome_service = webdriver.ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    return driver

In [6]:
def scrape_table_data(driver, url, proxy_manager):
    proxy = random.choice(proxy_manager.get_proxy_list())
    driver = setup_driver(proxy)
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tabProcedura")))
    links = driver.find_elements(By.XPATH, "//table[@id='tabProcedura']/tbody/tr/td[2]/a")
    data = []
    for link in links:
        link_url = link.get_attribute("href")
        link_text = link.text
        link.click()
        time.sleep(3)  # Adjust the delay as needed for the table to load
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        table = soup.find("table", id="elenco")
        headers = [th.text for th in table.find_all("th")]
        rows = []
        for row in table.find_all("tbody/tr"):
            row_data = [td.text for td in row.find_all("td")]
            rows.append(row_data)
        data.append({
            "Link": link_url,
            "Text": link_text,
            "Table Headers": headers,
            "Table Rows": rows
        })
        driver.back()
        time.sleep(3)  # Adjust the delay as needed
    return data

In [7]:
def main():
    proxy_manager = ProxyManager()
    url_base = "https://www.portalecreditori.it/procedure.php?filter=txtFAnno%7C2021&order=procedura&verso=asc&page="
    page_number = 1
    scraped_data = []

    while True:
        url = url_base + str(page_number)
        proxy = random.choice(proxy_manager.get_proxies())  # Get a random proxy for each page
        driver = setup_driver(proxy)
        driver.get(url)
        
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tabProcedura")))
            data = scrape_table_data(driver, url)
            scraped_data.extend(data)
            
            # Increment page number for the next iteration
            page_number += 1
        except Exception as e:
            print(f"Error occurred on page {page_number}: {str(e)}")
            break
        
        driver.quit()  # Close the driver after scraping each page
        time.sleep(random.uniform(1, 3))  # Add a random delay between requests to avoid being blocked

    # Convert scraped data to DataFrame
    df = pd.DataFrame(scraped_data)
    
    # Save DataFrame to CSV
    df.to_csv("scraped_data.csv", index=False)

if __name__ == "__main__":
    main()

TypeError: object of type 'NoneType' has no len()